### COMP5349 Cloud Computing Assignment 2

#### name

library loading

In [1]:
# Import all necessary libraries and setup the environment for matplotlib
%matplotlib inline
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
import numpy as np
import matplotlib.pyplot as plt
from time import time
from pyspark.sql.functions import *
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml import Pipeline


loading data

In [2]:
spark = SparkSession \
        .builder \
        .appName("COMP5349 Assignment 2") \
        .getOrCreate()

# train_datafile = "../MNIST/test_data/train_s.csv"
# train_labelfile = "../MNIST/test_data/train_label_s.csv"
# test_datafile = "../MNIST/test_data/test_s.csv"
# test_labelfile = "../MNIST/test_data/test_label_s.csv"

train_datafile = "/Users/allenzhang/Desktop/COMP5349 Cloud Computing/Assignment2/MNIST/test_data/train&label_s.csv"
test_datafile = "/Users/allenzhang/Desktop/COMP5349 Cloud Computing/Assignment2/MNIST/test_data/test&label_s.csv"

# train_datafile = "../MNIST/Train-label-28x28.csv"
# test_datafile = "../MNIST/Test-label-28x28.csv"


train_df = spark.read.csv(train_datafile, header=False,inferSchema="true")
test_df = spark.read.csv(test_datafile, header=False,inferSchema="true")

In [3]:
def assembler_df(df):
    assembler = VectorAssembler(
        inputCols=df.columns[1:], outputCol="features")
    df_vector = assembler.transform(df).select(
        col(df.columns[0]).alias("label"), "features")
    return df_vector

In [4]:
def pipline_assembler_pca(Spark, trainDat, testDat, k=5):
    # read data
    train_df = Spark.read.csv(trainDat, header=False, inferSchema="true")
    test_df = Spark.read.csv(testDat, header=False, inferSchema="true")

    # assembler them
    assembler = VectorAssembler(
        inputCols=train_df.columns[1:], outputCol="features")
    # PCA init
    pca = PCA(k=k, inputCol="features", outputCol="features_pcas")

    # pipeline set
    pipeline = Pipeline(stages=[assembler, pca])
    # fit model
    model = pipeline.fit(train_df)
    # transform train data
    train_pca_result = model.transform(train_df).select(
        col(train_df.columns[0]).alias("label"), "features_pcas")
    # transform test data
    test_pca_result = model.transform(test_df).select(
        col(test_df.columns[0]).alias("label"), "features_pcas")
    return train_pca_result, test_pca_result

In [5]:
train_vector_pca, test_vector_pca = pipline_assembler_pca(
    spark, train_datafile, test_datafile, k=50)

In [94]:
test_vector_pca.rdd.zipWithIndex().collect()

[(Row(label=7, features_pcas=DenseVector([517.8727, -581.9248, -185.5426, 911.9327, 47.8847, -312.0802, -400.8005, -310.7156, -749.1333, 125.9611, -384.4777, -279.4319, -280.7658, 238.8911, 69.5107, 85.9093, -2.1836, 117.1893, -0.9276, -153.0657, -40.7723, 228.6445, 74.8494, 176.1066, -106.2763, -63.6997, 105.2142, -202.9101, 39.977, -306.8406, 48.5439, -192.0236, 162.9929, 12.5452, -180.4817, 76.4046, 37.1126, -199.0994, 38.3111, -36.4913, 279.262, -15.2148, 96.984, -27.1398, -109.9488, -23.7491, 32.6208, -60.343, 29.0405, 154.4983])),
  0),
 (Row(label=2, features_pcas=DenseVector([686.8728, 942.4267, 158.9979, -252.505, -337.5044, -326.6398, -744.7112, 48.3965, -621.7675, 326.1669, -226.2866, -421.6629, -923.1901, 220.0115, -252.7706, 445.7953, -429.6566, 262.8321, 222.9663, -177.4741, -215.007, -230.5653, 120.3651, 423.2042, 133.1709, 110.2243, -151.1099, -156.2865, -234.1402, -0.9597, 130.6649, 102.193, -104.8245, 104.5235, -2.0674, 85.7815, 67.4907, 133.5543, 24.4065, 111.2329, -

In [6]:
cart_test_train = test_vector_pca.rdd.zipWithIndex().cartesian(train_vector_pca.rdd)

In [102]:
cart_test_train.take(1)

[((Row(label=7, features_pcas=DenseVector([517.8727, -581.9248, -185.5426, 911.9327, 47.8847, -312.0802, -400.8005, -310.7156, -749.1333, 125.9611, -384.4777, -279.4319, -280.7658, 238.8911, 69.5107, 85.9093, -2.1836, 117.1893, -0.9276, -153.0657, -40.7723, 228.6445, 74.8494, 176.1066, -106.2763, -63.6997, 105.2142, -202.9101, 39.977, -306.8406, 48.5439, -192.0236, 162.9929, 12.5452, -180.4817, 76.4046, 37.1126, -199.0994, 38.3111, -36.4913, 279.262, -15.2148, 96.984, -27.1398, -109.9488, -23.7491, 32.6208, -60.343, 29.0405, 154.4983])),
   0),
  Row(label=5, features_pcas=DenseVector([914.1563, 582.2488, -90.2267, 857.3614, -535.2763, -652.8056, -592.3889, -182.6522, -656.8593, 308.0764, -354.4112, -262.4242, 515.0922, -69.4853, -412.4822, 60.3366, -70.6442, -11.8284, 364.5197, 69.0636, 441.5512, -332.3223, -80.8986, -48.7875, 39.1037, 130.6269, -118.693, -142.5635, 241.2436, -136.0402, -90.5657, 109.7781, -127.2493, 66.6535, 119.2315, -158.2419, -32.7003, -317.0055, 49.4607, -250.442

In [7]:
def dist(record):
    test_record, train = record
    test, index = test_record
    dist = float(np.linalg.norm(test[1] - train[1]))
    return ((index, float(test[0])), (dist, float(train[0])))


In [8]:
dist_test_train = cart_test_train.map(dist)

In [9]:
dist_test_train.collect()

[((0, 7.0), (2194.34789057941, 5.0)),
 ((0, 7.0), (2490.909031062013, 0.0)),
 ((0, 7.0), (2035.2718445238934, 4.0)),
 ((0, 7.0), (2429.928254584547, 1.0)),
 ((0, 7.0), (2090.951897992186, 9.0)),
 ((0, 7.0), (2266.420072220303, 2.0)),
 ((0, 7.0), (2241.8618961437255, 1.0)),
 ((0, 7.0), (2466.891223387235, 3.0)),
 ((0, 7.0), (1975.7923078112913, 1.0)),
 ((0, 7.0), (2041.5961597215614, 4.0)),
 ((0, 7.0), (2258.158049069912, 3.0)),
 ((0, 7.0), (2167.9306383198173, 5.0)),
 ((0, 7.0), (2707.4550248257387, 3.0)),
 ((0, 7.0), (2480.501477758427, 6.0)),
 ((0, 7.0), (1992.6516812792797, 1.0)),
 ((0, 7.0), (1791.7482547851255, 7.0)),
 ((0, 7.0), (1978.8502669585494, 2.0)),
 ((0, 7.0), (2318.959591010781, 8.0)),
 ((0, 7.0), (2040.143827111917, 6.0)),
 ((0, 7.0), (1878.309260468727, 9.0)),
 ((0, 7.0), (2654.1767526267686, 4.0)),
 ((0, 7.0), (2629.410521536063, 0.0)),
 ((0, 7.0), (1930.4150437057026, 9.0)),
 ((0, 7.0), (2393.5513351808245, 1.0)),
 ((0, 7.0), (2194.3750627942563, 1.0)),
 ((0, 7.0), (

In [10]:
def mergeRecord(accumulatedPair, currentRecord):
    train_list = accumulatedPair
    train_list.append(currentRecord)
    train_list = sorted(train_list, key = lambda s: s[0], reverse=False)[:5]
    
    return train_list
    

In [11]:
def mergeCombiners(accumulate1, accumulate2):
    accumulate1.append(accumulate2)
    return accumulate1

In [12]:
ag = dist_test_train.aggregateByKey([(1e+10,0.0), (1e+10,0.0), (1e+10,0.0), (1e+10,0.0), (1e+10,0.0)], 
                                    mergeRecord, mergeCombiners, 1)

In [13]:
ag.collect()

[((4, 4.0),
  [(1009.9492617223102, 4.0),
   (1127.852285706814, 9.0),
   (1288.3426260909034, 4.0),
   (1288.472031502329, 4.0),
   (1322.5133049419069, 4.0)]),
 ((5, 1.0),
  [(550.5767693240937, 1.0),
   (561.3000352120652, 1.0),
   (620.1057053240866, 1.0),
   (696.6199058877445, 1.0),
   (724.1607639830399, 1.0)]),
 ((6, 4.0),
  [(1147.3571812778885, 4.0),
   (1153.0657483816317, 9.0),
   (1387.6271245861692, 9.0),
   (1414.3535004795583, 9.0),
   (1442.239927040776, 4.0)]),
 ((7, 9.0),
  [(1356.4374419580479, 9.0),
   (1420.64365259822, 9.0),
   (1463.8508315505342, 9.0),
   (1487.0148518886233, 9.0),
   (1531.7304493672348, 9.0)]),
 ((9, 9.0),
  [(1125.330107259811, 9.0),
   (1182.5280114743375, 7.0),
   (1205.0400341136017, 7.0),
   (1228.5831384489297, 9.0),
   (1246.9301731504593, 9.0)]),
 ((10, 0.0),
  [(1101.4013358721913, 0.0),
   (1256.7501670815222, 0.0),
   (1375.6992079408014, 0.0),
   (1382.7905659201872, 0.0),
   (1434.0814626828142, 0.0)]),
 ((15, 5.0),
  [(1274.4197

In [15]:
def selectLabel(record):
    keys, values = record
    topK_label = [x[1] for x in values]
    
    counts_test = np.bincount(topK_label)
    Most_test = np.argmax(counts_test)
    
    return Most_test

In [9]:
sel

TypeError: Invalid argument, not a string or column: 5 of type <class 'int'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [5]:
class knn_Spark(object):
    def __init__(self, Spark, train_data, k_knn=5, distance="euclidean"):
        global trainLab
        global trainFea
        global K_knn
        global Dist

        # broadcast label of train data
        # broadcast features of train data
        list_trainDat = train_data.collect()
        trainLab = Spark.sparkContext.broadcast(
            np.array([x[0] for x in list_trainDat]))
        trainFea = Spark.sparkContext.broadcast(
            np.array([x[1] for x in list_trainDat]))

        K_knn = k_knn
        Dist = distance

    @staticmethod
    def knn_spark_map(features):
        # label, features = record
        # eu distance with numpy
        if Dist == "euclidean":
            dist = np.linalg.norm(
                trainFea.value - np.array(features), axis=1).reshape(trainLab.value.shape)

        # manhattan distance with numpy
        elif Dist == "manhattan":
            dist = np.abs(trainFea.value - np.array(features)
                          ).sum(axis=1).reshape(trainLab.value.shape)

        # correlation with numpy
        elif Dist == "correlation":
            data_norm1 = ((np.array(features) - np.mean(np.array(features))) /
                          np.std(np.array(features)))

            data_norm2 = (trainFea.value - trainFea.value.mean(axis=1).reshape(len(
                trainFea.value), 1)) / trainFea.value.std(axis=1).reshape(len(trainFea.value), 1)
            dist = -np.sum((data_norm1 * data_norm2 / float(
                np.array(features).shape[0])), axis=1).reshape(trainLab.value.shape)

        # chebyshev distance with numpy
        elif Dist == "chebyshev":
            dist = np.abs(trainFea.value - np.array(features)
                          ).max(axis=1).reshape(trainLab.value.shape)

        # new knn method no concatenate
        topK_label = np.take(
            trainLab.value, np.argpartition(dist, K_knn, axis=0)[:K_knn])

        counts_test = np.bincount(topK_label)
        Most_test = np.argmax(counts_test)
        return float(Most_test)

    def predict(self, test_data):
        func = udf(self.knn_spark_map)
        # return test_data.rdd.map(self.knn_spark_map)
        return test_data.withColumn('test', func(test_data.columns[1]))

In [16]:
model = knn_Spark(spark, train_vector_pca, k_knn=5, distance="euclidean")
res = model.predict(test_vector_pca)

In [17]:
result = res.select('test', 'label')

In [ ]:
def f(test, real):
    equal_tt = 0
    lentest = 0
    if test == real:
        equal_tt += 1
    
    lentest += 1
    
    return equal_tt / lentest

In [11]:
import pyspark

In [26]:
f = udf(lambda x: type(x))

In [28]:
res_precision = result.groupby("test").agg(collect_list("label").alias("label_list")) \
        .withColumn("precision", f("label_list")).show()



+----+--------------------+--------------------+
|test|          label_list|           precision|
+----+--------------------+--------------------+
| 1.0|[1, 1, 1, 1, 1, 1...|net.razorvine.pic...|
| 0.0|[0, 0, 0, 0, 0, 6...|net.razorvine.pic...|
| 9.0|[4, 9, 9, 9, 9, 4...|net.razorvine.pic...|
| 5.0|        [5, 3, 5, 5]|net.razorvine.pic...|
| 6.0|[6, 6, 6, 6, 6, 6...|net.razorvine.pic...|
| 4.0|[4, 5, 4, 4, 4, 4...|net.razorvine.pic...|
| 7.0|[7, 7, 7, 7, 7, 7...|net.razorvine.pic...|
| 2.0|[2, 2, 2, 2, 8, 2...|net.razorvine.pic...|
| 8.0|                 [8]|net.razorvine.pic...|
| 3.0|[5, 3, 3, 3, 3, 5...|net.razorvine.pic...|
+----+--------------------+--------------------+



----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 65088)
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_upda

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:65071)
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_i

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:65071)
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_i

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:65071)
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_i

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:65071)
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_i

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:65071)
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_i

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:65071)
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_i

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:65071)
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_i

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:65071)
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_i

In [22]:
result.show()

+----+-----+
|test|label|
+----+-----+
| 7.0|    7|
| 2.0|    2|
| 1.0|    1|
| 0.0|    0|
| 4.0|    4|
| 1.0|    1|
| 9.0|    4|
| 9.0|    9|
| 4.0|    5|
| 9.0|    9|
| 0.0|    0|
| 6.0|    6|
| 9.0|    9|
| 0.0|    0|
| 1.0|    1|
| 3.0|    5|
| 9.0|    9|
| 7.0|    7|
| 3.0|    3|
| 9.0|    4|
+----+-----+
only showing top 20 rows



In [ ]:
result.persist(pyspark.StorageLevel.MEMORY_ONLY)

In [18]:
res2=result.persist(pyspark.StorageLevel.MEMORY_ONLY)

In [ ]:
result

In [8]:
def evaluate_precision_recall_fscore(resultDf):
    print(111111111111111111333333333333333333)
    resultDf.show()
    print("\n\n\n11111111111111111111111113333333333333333")
    col_p = "test"
    col_l = "label"
    precisionF = udf(lambda pred, lab_list:
                     np.round(((np.array(lab_list) == float(pred)).sum().tolist()) / len(lab_list) * 100, 2).tolist())

    res_precision = resultDf.groupby(col_p).agg(collect_list(col_l).alias("label_list")) \
        .withColumn("precision", round(precisionF(col_p, 'label_list'), 2)).select(col_p, "precision")

    res_precision.show()

    recallF = udf(lambda pred_list, lab:
                  np.round((np.array(pred_list, dtype=float) == lab).sum() / len(pred_list) * 100, 2).tolist())

    res_recall = resultDf.groupby(col_l).agg(collect_list(col_p).alias("predict_list")) \
        .withColumn("recall", round(recallF("predict_list", col_l), 2)).select(col_l, "recall")

    res_recall.show()

    fscoreF = udf(lambda precision, recall: (
        (2 * float(precision) * float(recall)) / (float(precision) + float(recall))))

    evaluation_result = res_precision.join(res_recall, res_precision.test == res_recall.label) \
        .select(col_p, "precision", "recall") \
        .withColumn("F-Measure", round(fscoreF("precision", "recall"), 2))

    return evaluation_result

In [19]:
a = evaluate_precision_recall_fscore(res2)

111111111111111111333333333333333333
+----+-----+
|test|label|
+----+-----+
| 7.0|    7|
| 2.0|    2|
| 1.0|    1|
| 0.0|    0|
| 4.0|    4|
| 1.0|    1|
| 9.0|    4|
| 9.0|    9|
| 4.0|    5|
| 9.0|    9|
| 0.0|    0|
| 6.0|    6|
| 9.0|    9|
| 0.0|    0|
| 1.0|    1|
| 3.0|    5|
| 9.0|    9|
| 7.0|    7|
| 3.0|    3|
| 9.0|    4|
+----+-----+
only showing top 20 rows




11111111111111111111111113333333333333333
+----+---------+
|test|precision|
+----+---------+
| 1.0|     87.5|
| 0.0|     80.0|
| 9.0|    64.29|
| 5.0|     75.0|
| 6.0|    100.0|
| 4.0|    84.62|
| 7.0|    93.33|
| 2.0|     87.5|
| 8.0|    100.0|
| 3.0|    81.82|
+----+---------+

+-----+------+
|label|recall|
+-----+------+
|    1| 100.0|
|    6|  80.0|
|    3| 81.82|
|    5| 42.86|
|    9| 81.82|
|    4| 78.57|
|    8|  50.0|
|    7| 93.33|
|    2|  87.5|
|    0| 100.0|
+-----+------+



In [20]:
a.show()

+----+---------+------+---------+
|test|precision|recall|F-Measure|
+----+---------+------+---------+
| 1.0|     87.5| 100.0|    93.33|
| 0.0|     80.0| 100.0|    88.89|
| 9.0|    64.29| 81.82|     72.0|
| 5.0|     75.0| 42.86|    54.55|
| 6.0|    100.0|  80.0|    88.89|
| 4.0|    84.62| 78.57|    81.48|
| 7.0|    93.33| 93.33|    93.33|
| 2.0|     87.5|  87.5|     87.5|
| 8.0|    100.0|  50.0|    66.67|
| 3.0|    81.82| 81.82|    81.82|
+----+---------+------+---------+



In [ ]:
def f1(test, label):
    
    b = (np.array(label) == float(test)).sum().tolist()
    
    c = np.round(b / len(label) * 100, 2).tolist()
    
    return c
    

In [ ]:
ff1 = udf(f1)

In [23]:
precisionF = udf(lambda pred, lab_list:
                     np.round(((np.array(lab_list) == float(pred)).sum().tolist()) / len(lab_list) * 100, 2).tolist())


In [24]:
t1 = res2.groupby('test').agg(collect_list("label").alias("label_list")) \
.withColumn('precision',precisionF('test','label_list')).select('test', 'precision')

In [25]:
t1.show()

+----+---------+
|test|precision|
+----+---------+
| 1.0|     87.5|
| 0.0|     80.0|
| 9.0|    64.29|
| 5.0|     75.0|
| 6.0|    100.0|
| 4.0|    84.62|
| 7.0|    93.33|
| 2.0|     87.5|
| 8.0|    100.0|
| 3.0|    81.82|
+----+---------+



In [ ]:
def f2(test_list, label):
    b = (np.array(test_list, dtype = float) == label).sum()
    c = np.round(b / len(test_list) * 100, 2).tolist()
    return c
#     return test_list1
ff2 = udf(f2)

In [ ]:
recallF = udf(lambda pred_list, lab:
                  np.round((np.array(pred_list, dtype=float) == lab).sum() / len(pred_list) * 100, 2).tolist())


In [ ]:
t2 = res2.groupby('label').agg(collect_list('test').alias('test_list')) \
.withColumn("recall", recallF('test_list', 'label')).select('label', "recall")

In [ ]:
t2.show()

In [ ]:
   fscoreF = udf(lambda precision, recall: (
        (2 * float(precision) * float(recall)) / (float(precision) + float(recall))))


In [ ]:
a = t1.join(t2, t1.test == t2.label).select("test", 'precision', "recall") \
.withColumn('F-Measure', round(fscoreF('precision', "recall"), 2))

In [ ]:
a.show()

In [ ]:
s1 = time()
t1.join(t2, t1.test == t2.label).select("test", 'precision', "recall") \
.withColumn('F-Measure', round(
    (2 * col("precision") * col("recall")) / (col("precision") + col("recall")), 2)).show()
s2 = time()
ss = s2 -s1

In [ ]:
ss

In [ ]:
sc = spark.sparkContext

In [ ]:
df = sc.parallelize([(10100720363468236,["what", "sad", "to", "me"]),
                     (10100720363468236,["what", "what", "does", "the"]),
                     (10100718890699676,["at", "the", "oecd", "with"])]).toDF(["id", "message"])

In [ ]:
from pyspark.sql.types import *
from collections import Counter

# We need to specify the schema of the return object
schema_count = ArrayType(StructType([
    StructField("word", StringType(), False),
    StructField("count", IntegerType(), False)
]))

count_udf = udf(
    lambda s: Counter(s).most_common(), 
    schema_count
)

unpack_udf = udf(
    lambda l: [item for sublist in l for item in sublist]
)

In [ ]:
aa = df.groupBy("id").agg(collect_list("message").alias("message"))  \
# .withColumn("message", unpack_udf("message")) \
# .withColumn("message", count_udf("message"))

In [ ]:
aa.select('message').show(1)

In [ ]:
metrics = MulticlassMetrics(res)

In [ ]:
print("The accuracy is {0:.4f}".format(metrics.accuracy))

In [ ]:
np.sum(confMatrix[1])

In [ ]:
confMatrix = metrics.confusionMatrix().toArray()

In [ ]:
def get_statistics_for_class(confusion_matrix, label):
    TP = confusion_matrix[label, label]
    FN = np.sum(confusion_matrix[label]) - TP
    FP = np.sum(confusion_matrix[:, label]) - TP
    TN = np.sum(confusion_matrix) - TP - FN - FP
    return TP, TN, FP, FN

In [ ]:
def precision (TP, TN, FP, FN):
    return TP / (TP + FP)

def recall (TP, TN, FP, FN):
    return TP / (TP + FN)

def F_measure (TP, TN, FP, FN):
    return 2*TP / (2*TP + FP + FN)

def accuracy (TP, TN, FP, FN):
    return (TP + TN) / (TP + TN + FP + FN)

In [ ]:
def make_statistics(confusion_matrix, full_class_list):
    # Determine statistical parameters precision, recall, F-measure from TP and TN
    class_count = len(full_class_list)
    stats = np.zeros((class_count, 4))
    for i in range(class_count):
        TP, TN, FP, FN = get_statistics_for_class (confusion_matrix, i)
        stats[i,0] = np.round(precision(TP, TN, FP, FN),2)
        stats[i,1] = np.round(recall(TP, TN, FP, FN),2)
        stats[i,2] = np.round(F_measure(TP, TN, FP, FN),2)
        stats[i,3] = np.round(accuracy(TP, TN, FP, FN),2)
    return stats

In [ ]:
a = make_statistics(confMatrix, label)

In [ ]:
np.array([x[3] for x in a]).mean()

In [ ]:
from pyspark.sql import Row

In [ ]:
R = Row('label', 'precision', 'recall', 'F_measure', 'accuracy')

In [ ]:
c = [Row(float(x[0]), float(x[1]), float(x[2]), 
       float(x[3]), float(x[4])) for x in b]

In [ ]:
c

In [ ]:
    schema = StructType([
        StructField("label", FloatType(), True),
        StructField("precision", FloatType(), True),
        StructField("recall", FloatType(), True),
        StructField("F_measure", FloatType(), True),
        StructField("accuracy", FloatType(), True),
    ])

In [ ]:
spark.createDataFrame(c, schema).show()

In [ ]:
train_vector_pca.show()

In [ ]:
train_vector_pca

In [ ]:
from pyspark.ml.classification import *

In [ ]:
mlp = MultilayerPerceptronClassifier(labelCol="label", featuresCol="features_pcas", 
                                     maxIter=100, layers=[5, 15, 10], blockSize=32)

In [ ]:
model = mlp.fit(train_vector_pca)

In [ ]:
pipeline = Pipeline(stages=[test_vector_pca, model])

In [ ]:
model = pipeline.transform(test_vector_pca)

In [ ]:
model.transform(test_vector_pca).select("label", "prediction").show()

In [ ]:
label = np.array(range(0,10))

In [ ]:
for i in range(10):
    m_recal.append(metrics.recall(i))
    

In [ ]:
print("{} b".format(m_recal))

In [ ]:
def f1_score_single(y_true, y_pred):
    y_true = set(y_true)
    y_pred = set(y_pred)
    cross_size = len(y_true & y_pred)
    if cross_size == 0: return 0.
    p = 1. * cross_size / len(y_pred)
    r = 1. * cross_size / len(y_true)
    return 2 * p * r / (p + r)

In [ ]:
# broadcast the train data
# init knn_model with train data
model = knn_Spark(spark, train_vector_pca, k_knn=5)
res = model.fit(test_vector_pca)

In [ ]:
res.()

In [ ]:
res

In [ ]:
a = res.foreach(lambda x: print(x))

In [ ]:
global test

test = []
def f(record):
    test.append(record)


In [ ]:

res.map(f)

In [ ]:
sc = spark.sparkContext

In [ ]:
sc.parallelize([(1, 2) , (3, 4), (5,6)])

In [ ]:
spark.stop()